In [ ]:
# install required packages
install.packages(c("leaflet", "sf", "jsonlite", "httr", "shiny"))

In [ ]:
# load packages
library("leaflet")
library("sf")
library("jsonlite")
library("httr")
library("shiny")

In [ ]:
# define maximum features
maxfeatures <- 2000

# create empty data frame
df_total <- data.frame()

# input parameters for bbox and key
bbox <- "-3.545247,50.725949,-3.538380,50.729229"
key <- "INSERT_API_KEY_HERE"
collection_name <- "bld-fts-buildingpart"

# call Maps API for basemapping
OSLight_3857 <- paste0("https://api.os.uk/maps/raster/v1/zxy/Light_3857/{z}/{x}/{y}.png?key=", key)

# return json data from API
url <- paste0("https://api.os.uk/features/ngd/ofa/v1/collections/", collection_name, "/items?bbox=", bbox)
rawdata <- GET(url, add_headers(key=key))
json <- fromJSON(rawToChar(rawdata$content), flatten=TRUE)
sf <- read_sf(GET(url, add_headers(key=key)), as_tibble=FALSE)

In [ ]:
# add first 100 features from API response to data frame
df_total = rbind(df_total, sf)

# page through API binding to data frame until maximum features reached, or all features collected
while (TRUE) {
    if ("Next page" == tail(json$links$title, n=1)&maxfeatures>(nrow(df_total))) {
        url <- (tail(json$links$href, n=1))
        rawdata <- GET(url, add_headers(key=key))
        json <- fromJSON(rawToChar(rawdata$content), flatten=TRUE)
        sf <- read_sf(GET(url, add_headers(key=key)), as_tibble=FALSE)
        df_total <- rbind(df_total, sf)
    }
    else {
        break
    }
}

In [ ]:
# define app server functions for shiny 
server <- function(input, output, session) {
    output$map <- renderLeaflet({
        m <- leaflet() %>%
        addTiles(OSLight_3857) %>%
        setView(lng=-3.541809, lat=50.727589, zoom=17) %>%
        addPolygons(data=df_total, color="#3388ff", weight=1, opacity=1.0, fillOpacity=0.5)
    })
}

# define app front-end
ui <- fluidPage(
    leafletOutput("map", height=600)
)

# run app
shinyApp(ui, server)